Diagnosing Student Code with Avicenna
===

In this Jupyter Notebook, we are exploring an interesting problem in educational technology and computer science education: the automated evaluation and diagnosis of student-submitted code. While automated systems can easily evaluate the correctness of code, diagnosing the _reasons_ behind any incorrect behavior can be a complex task.

### The Problem: Greatest Common Divisor (GCD)

Students have been assigned the task of writing a Python function to calculate the Greatest Common Divisor (GCD) of two integers. The GCD of two integers is the largest integer that divides both numbers without a remainder.

### Avicenna: Our Debugging and Diagnosis Tool

To aid us in this endeavor, we will be using Avicenna, an automated debugging tool that uses fuzz testing and symbolic reasoning to identify the conditions under which code behaves incorrectly. Avicenna's goal is to automatically diagnose software faults with high precision and recall, thereby making the debugging process more efficient.

With this notebook, we aim to:

1. Showcase how Avicenna can be used to diagnose student code.
2. Illustrate the capabilities of Avicenna through an example of diagnosing a GCD implementation.
3. Demonstrate how the diagnosis output can be used for generating new failure-inducing inputs.

By the end of this notebook, you'll have a clearer understanding of how automated diagnosis can aid in educational settings, and how tools like Avicenna contribute to this.

### Student Implementation

Below is the student's GCD implementation in Python.

In [1]:
# Student's implementation for calculating GCD
class Solution:
    def gcd(self, A, B):
        # code here
        l1=[]
        for i in range(1,max(A,B)):
            if A%i==0 and B%i==0:
                l1.append(i)
        return max(l1)

### Reference Implementation

This is a reference implementation for calculating the GCD using the Euclidean algorithm.

In [2]:
# Reference implementation using the Euclidean algorithm
class Reference():
    def gcd(self, A, B):
        if A < B:
            A, B = B, A
        while B:
            A, B = B, A % B
        return A

### Testing the Implementations

Let's run some initial tests to compare the student and reference implementations.

In [3]:
# Comparing the two implementations
ob = Solution()
print(ob.gcd(10, 10))  # Output should be 10 for correct implementation

ref = Reference()
print(ref.gcd(10, 10))  # Output should be 10 as well

5
10


### Fuzzing Setup

Here we set up a grammar for fuzz testing using GrammarFuzzer.

In [4]:
# Setting up a grammar for fuzzing
from fuzzingbook.Grammars import Grammar
import string 

grammar: Grammar = {
    '<start>': ["<input>"],
    "<input>": ["<first> <second>"],
    "<first>": ["<integer>"],
    "<second>": ["<integer>"],
    "<integer>": ["<onenine><maybe_digits>"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
}

### Oracle Construction

We will now use the reference implementation to construct an oracle for fuzz testing.

In [5]:
# Constructing an oracle using the reference GCD implementation
from avicenna.oracle_construction import construct_oracle, UnexpectedResultError, OracleResult

error_def = {UnexpectedResultError: OracleResult.FAILING}
oracle = construct_oracle(ob.gcd, ref.gcd, error_def)

### Running the Fuzzer

Let's run the fuzzer and examine the results.

In [6]:
# Running the fuzzer to test the student's implementation
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

fuzzer = GrammarFuzzer(grammar)
for _ in range(10):
    inp = fuzzer.fuzz()
    print(inp.ljust(30), oracle(inp))

455 873                        PASSING
7564 69922                     PASSING
3 56                           PASSING
5 4                            PASSING
34 764                         PASSING
962439 5                       PASSING
8 7                            PASSING
7 9                            PASSING
95 68                          PASSING
6 71                           PASSING


In [7]:
from avicenna.input import Input
oracle(Input.from_str(grammar, "4 4"))

<OracleResult.FAILING: 'FAILING'>

### Diagnosing Failures with Avicenna

We will now use Avicenna to automatically diagnose the conditions under which the student's implementation fails.

In [8]:
# Using Avicenna for diagnosis
from avicenna import Avicenna

initial_inputs = ["10 2", "4 4", "49 6618904968"]
avicenna = Avicenna(
    grammar,
    initial_inputs=initial_inputs,
    oracle=oracle
)

diagnosis = avicenna.explain()

∀ elem ∈ start: (StrToInt(elem) >= StrToInt(STRING_0))
∀ container ∈ start: (∃ elem ∈ container: (Length(elem) == StrToInt(STRING_0)))
∀ elem ∈ start: (StrToInt(elem) <= StrToInt(STRING_0))
∃ container1 ∈ start: (∃ length_field ∈ start: (Length(container1) < StrToInt(length_field)))
∃ elem ∈ start: (elem == STRING_0)
∀ container ∈ start: (∃ elem ∈ container: (elem == STRING_0))
∀ elem_1 ∈ start: (∃ elem_2 ∈ start: (StrToInt(elem_1) > StrToInt(elem_2)))
∃ container ∈ start: (∃ length_field ∈ start: (Length(container) <= StrToInt(length_field)))
Number of candidates:  27
20 20
Number of candidates:  27
47 47
Number of candidates:  27
22 22
Number of candidates:  27
22 22
Number of candidates:  27
22 22
Number of candidates:  16
31 31
Number of candidates:  16
33 33
Number of candidates:  16
31 31
Number of candidates:  27
33 33
Number of candidates:  27
33 33


### Viewing the Diagnosis

Let's interpret the diagnosis provided by Avicenna.

In [9]:
# Printing and interpreting the diagnosis
from isla.language import ISLaUnparser

if diagnosis:
    print(f"Avicenna determined the following constraints to describe the failure circumstances:\n")
    print(ISLaUnparser(diagnosis[0]).unparse())
    print(f"Avicenna calculated a precision of {diagnosis[1]*100:.2f}% and a recall of {diagnosis[2]*100:.2f}%", end="\n\n")
else:
    print("Avicenna was not able to generate a diagnosis!")

Avicenna determined the following constraints to describe the failure circumstances:

(forall <second> container in start:
   exists <integer> elem in container:
     (= (str.len elem) (str.to.int "1")) and
exists <first> container1 in start:
  exists <second> length_field in start:
    (< (str.len container1) (str.to.int length_field)))
Avicenna calculated a precision of 67.52% and a recall of 100.00%



## Use Self-Defined Patterns

In [10]:
error_def = {UnexpectedResultError: OracleResult.FAILING}
oracle = construct_oracle(ob.gcd, ref.gcd, error_def)

In [11]:
patterns = [
"""exists <?NONTERMINAL> elem_1 in start:
  exists <?NONTERMINAL> elem_2 in start:
    (= (str.to.int elem_1) (str.to.int elem_2))""",
    """
    exists <?NONTERMINAL> elem in start:
    (>= (str.to.int elem) (str.to.int <?STRING>))
    """
]

In [12]:
# Using Avicenna for diagnosis
from avicenna import Avicenna

avicenna = Avicenna(
    grammar,
    initial_inputs=initial_inputs,
    oracle=oracle,
    patterns=patterns,
    max_iterations=20,
    top_n_relevant_features=3
)

diagnosis = avicenna.explain()

∃ elem_1 ∈ start: (∃ elem_2 ∈ start: (StrToInt(elem_1) == StrToInt(elem_2)))
∃ elem ∈ start: (StrToInt(elem) >= StrToInt(STRING_0))
Number of candidates:  8
8 8
Number of candidates:  8
8 8
Number of candidates:  8
6 6
Number of candidates:  8
6 6
Number of candidates:  8
6 6
Number of candidates:  8
10 10
Number of candidates:  8
12 12
Number of candidates:  8
12 12
Number of candidates:  8
12 12
Number of candidates:  8
12 12
Number of candidates:  8
12 12
Number of candidates:  8
12 12
Number of candidates:  8
12 12
Number of candidates:  8
16 16
Number of candidates:  8
16 16
Number of candidates:  8
16 16
Number of candidates:  8
16 16
Number of candidates:  8
18 18
Number of candidates:  8
18 18
Number of candidates:  3
18 18


In [13]:
# Printing and interpreting the diagnosis
from isla.language import ISLaUnparser

if diagnosis:
    print(f"Avicenna determined the following constraints to describe the failure circumstances:\n")
    print(ISLaUnparser(diagnosis[0]).unparse())
    print(f"Avicenna calculated a precision of {diagnosis[1]*100:.2f}% and a recall of {diagnosis[2]*100:.2f}%", end="\n\n")
else:
    print("Avicenna was not able to generate a diagnosis!")

Avicenna determined the following constraints to describe the failure circumstances:

exists <second> elem_1 in start:
  exists <first> elem_2 in start:
    (= (str.to.int elem_1) (str.to.int elem_2))
Avicenna calculated a precision of 100.00% and a recall of 100.00%

